In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.3 MB/s eta 0:00:00


In [2]:
import torch
import networkx as nx

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [ ]:
class GCN(torch.nn.Module):
    def __init__(
        self,
        ...
        conv_layer = "LGC",
        ...
    ):
        ...
        # initialize convolutional layers
        self.conv_layer = conv_layer
        if conv_layer == "LGC":
          self.convs = ModuleList([LGConv(**kwargs) for _ in range(num_layers)])
        elif conv_layer == "GAT": # initialize GAT layer with multiple heads
          n_heads = 5
          self.convs = ModuleList([
              GATConv(embedding_dim, embedding_dim, heads = n_heads,
                      dropout = 0.5, **kwargs) for _ in range(num_layers)
          ])
          self.linears = ModuleList([Linear(n_heads * embedding_dim,
                                     embedding_dim) for _ in range(num_layers)])

        elif conv_layer == "SAGE": #  initialize GraphSAGE conv
          self.convs = ModuleList([
              SAGEConv(embedding_dim, embedding_dim, **kwargs)
              for _ in range(num_layers)
          ])

SyntaxError: invalid syntax (ipython-input-3523650860.py, line 4)

In [ ]:
class LGConv(MessagePassing):
    def __init__(self, **kwargs):
        super(LGConv, self).__init__(**kwargs)

    def forward(self, x, edge_index, size = None):
        # calculate normalization term (in the above: 1 / sqrt(...))
        src, dst = edge_index
        node_degree = torch_geometric.utils.degree(dst)
        node_degree_inv_sqrt = node_degree.pow(-0.5)
        norm = node_degree_inv_sqrt[src] * node_degree_inv_sqrt[dst]
        # pass to propagate (returns the term on the RHS in above)
        out = self.propagate(edge_index, x = x, norm = norm, size = size)
        return out

    def message(self, x_j, norm):
        # calculating normalized message (term in the sum above)
        return x_j * norm.view(-1, 1)

    # Note: could also just pass aggr = 'add' to __init__()
    def aggregate(self, inputs, index, dim_size = None):
        # aggregation (the sum above)
        out = torch_scatter.scatter(inputs, index, dim=self.node_dim, reduce='sum')
        return out

NameError: name 'MessagePassing' is not defined